In [ ]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random
directories = [
    'C:/Users/hp/Downloads/music/Data',
    '/content/spectrograms3sec/train',
    '/content/spectrograms3sec/test'
]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

genres = ['blues', 'classical', 'country', 'disco', 'pop', 'hiphop','metal', 'reggae', 'rock']

for g in genres:
    path_audio = os.path.join('/content/audio3sec', f'{g}')
    if not os.path.exists(path_audio):
        os.makedirs(path_audio)
    path_train = os.path.join('/content/gdrive/My Drive/spectrograms3sec/train', f'{g}')
    if not os.path.exists(path_train):
        os.makedirs(path_train)
    path_test = os.path.join('/content/gdrive/My Drive/spectrograms3sec/test', f'{g}')
    if not os.path.exists(path_test):
        os.makedirs(path_test)


from pydub import AudioSegment
i = 0
for g in genres:
  j=0
  print(f"{g}")
  for filename in os.listdir(os.path.join('C:\\Users\\hp\\Downloads\\music\\Data\\genres_original',f"{g}")):

    song  =  os.path.join(f'C:\\Users\\hp\\Downloads\\music\\Data\\genres_original',f"{g}",f'{filename}')
    j = j+1
    for w in range(0,10):
      i = i+1
      #print(i)
      t1 = 3*(w)*1000
      t2 = 3*(w+1)*1000
      newAudio = AudioSegment.from_wav(song)
      new = newAudio[t1:t2]
      new.export(f'/content/audio3sec/{g}/{g+str(j)+str(w)}.wav', format="wav")

import os
genres = ['blues', 'classical', 'country', 'disco', 'pop', 'hiphop','metal', 'reggae', 'rock']
for g in genres:
  j = 0
  print(g)
  directory=rf'C:\\Users\\hp\\Downloads\\music\\Data\\spectograms3sec//{g}'
  os.makedirs(directory,exist_ok=True)
  for filename in os.listdir(os.path.join('C:\\Users\\hp\\Downloads\\music\\Data\\genres_original',f"{g}")):
    song  =  os.path.join(f'C:\\Users\\hp\\Downloads\\music\\Data\\genres_original\\{g}',f'{filename}')
    j = j+1
    
    y,sr = librosa.load(song,duration=3)
    #print(sr)
    mels = librosa.feature.melspectrogram(y=y,sr=sr)
    fig = plt.Figure()
    canvas = FigureCanvas(fig)
    p = plt.imshow(librosa.power_to_db(mels,ref=np.max))
    plt.savefig(f'C:\\Users\\hp\\Downloads\\music\\Data\\spectograms3sec//{g}//{g+str(j)}.png')

import os
import shutil
from sklearn.model_selection import train_test_split

test_size = 0.2   
random_state = 42  

for g in genres:
    train_dir = f'C:/Users/hp/Downloads/music/Data/spectograms3sec/train/{g}'
    test_dir = f'C:/Users/hp/Downloads/music/Data/spectrograms3sec/test/{g}'
    
    os.makedirs(test_dir, exist_ok=True)

    files = os.listdir(train_dir)

    train_files, test_files = train_test_split(files, test_size=test_size, random_state=random_state)

    for file in test_files:
        src = os.path.join(train_dir, file)
        dst = os.path.join(test_dir, file)
        shutil.move(src, dst)

print("Data split complete.")


train_dir = "C:/Users/hp/Downloads/music/Data/spectograms3sec/train/"
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(288,432),color_mode="rgba",class_mode='categorical',batch_size=128)

validation_dir = "C:/Users/hp/Downloads/music/Data/spectrograms3sec/test/"
vali_datagen = ImageDataGenerator(rescale=1./255)
vali_generator = vali_datagen.flow_from_directory(validation_dir,target_size=(288,432),color_mode='rgba',class_mode='categorical',batch_size=128)


def GenreModel(input_shape = (288,432,4),classes=9):
  
  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(3,3),strides=(1,1))(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(16,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(32,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(128,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  
  X = Flatten()(X)
  
  X = Dropout(rate=0.3)

  X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

  model = Model(inputs=X_input,outputs=X,name='GenreModel')

  return model

classes = len(genres)
from keras.layers import Dropout
import keras.backend as K

def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

X_input = Input(shape=(288, 432, 4), name='input_layer')
X = X_input

# Rest of your model architecture

X = Flatten()(X)
X = Dropout(rate=0.3)(X)  # Apply Dropout layer after Flatten

X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

model = Model(inputs=X_input, outputs=X, name='GenreModel')

opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', get_f1])

model.fit_generator(train_generator, epochs=70, validation_data=vali_generator)


import matplotlib.pyplot as plt

history = model.fit_generator(train_generator, epochs=70, validation_data=vali_generator)

accuracy = history.history['accuracy']
loss = history.history['loss']
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


model.save('C:\\Users\\hp\\Downloads\\music')


import tensorflow.keras.models as models

# Define the custom object
class MeanMetricWrapper:
    def __init__(self, metric):
        self.metric = metric

    def __call__(self, *args, **kwargs):
        return self.metric(*args, **kwargs)

# Define your custom metric function
def get_f1(y_true, y_pred):
    # Implementation of your custom metric
    # ...

# Load the model with the custom object
       model = models.load_model('C:\\Users\\hp\\Downloads\\music',     custom_objects={'MeanMetricWrapper': MeanMetricWrapper(get_f1)    }       )    

# Save the model again
model.save('C:\\Users\\hp\\Downloads\\music_updated')


import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Define the custom object
class MeanMetricWrapper:
    def __init__(self, metric):
        self.metric = metric

    def __call__(self, *args, **kwargs):
        return self.metric(*args, **kwargs)

# Define your custom metric function
def get_f1(y_true, y_pred):
    # Implementation of your custom metric
    # ...

# Load the trained model with the custom object
  model = tf.keras.models.load_model('C:\\Users\\hp\\Downloads\\music_updated', custom_objects={'MeanMetricWrapper': MeanMetricWrapper(get_f1)})

# Set the path to the test data directory
test_data_dir = 'C:/Users/hp/Downloads/music/Data/spectrograms3sec/test/'

# Get the list of genres
genres = ['blues', 'classical', 'country', 'disco', 'pop', 'hiphop', 'metal', 'reggae', 'rock']

# Iterate over the test data for each genre
for genre in genres:
    genre_dir = os.path.join(test_data_dir, genre)
    genre_files = os.listdir(genre_dir)

    correct_predictions = 0
    total_predictions = 0

    # Iterate over each test image in the genre directory
    # Iterate over the test data for each genre
for genre in genres:
    genre_dir = os.path.join(test_data_dir, genre)
    genre_files = os.listdir(genre_dir)

    correct_predictions = 0
    total_predictions = 0

    for filename in genre_files:
        img_path = os.path.join(genre_dir, filename)
        img = image.load_img(img_path, target_size=(288, 432))
        img_array = image.img_to_array(img)
        
        # Add an alpha channel to the image
        img_array = np.concatenate([img_array, np.zeros((img_array.shape[0], img_array.shape[1], 1))], axis=-1)
        
        img_array = np.expand_dims(img_array, axis=0)
        preprocessed_img = img_array / 255.0  # Normalize the image

        # Make prediction
        predictions = model.predict(preprocessed_img)
        predicted_class = np.argmax(predictions)
        
        # Print the predicted class and compare with the ground truth
        print('Predicted class:', predicted_class, 'Ground truth:', genres.index(genre))
        if predicted_class == genres.index(genre):
            correct_predictions += 1
        total_predictions += 1

    accuracy = correct_predictions / total_predictions
    print('Genre:', genre, 'Accuracy:', accuracy)

import numpy as np
import tensorflow as tf
import librosa
from scipy import signal

# Load the audio file
audio_path = 'C:\\Users\\hp\\Downloads\\nsqiv-0otrm.wav'
audio, sr = librosa.load(audio_path, sr=None)

# Preprocess the audio data to generate a spectrogram
# You can adjust the parameters according to your requirements
window_size = 1024
hop_length = 512
n_fft = 1024

_, _, spectrogram = signal.spectrogram(audio, fs=sr, window='hann', nperseg=window_size, noverlap=window_size-hop_length, nfft=n_fft)

# Normalize the spectrogram
spectrogram = np.log1p(spectrogram)

# Resize the spectrogram to match the input shape of the model
input_shape = (spectrogram.shape[0], spectrogram.shape[1], 1)
spectrogram = tf.image.resize(spectrogram, input_shape[:-1])

# Expand dimensions to match the batch size expected by the model
spectrogram = np.expand_dims(spectrogram, axis=0)

# Load the model
model_path = 'C:\\Users\\hp\\Downloads\\music_updated'
model = tf.keras.models.load_model(model_path)

# Make predictions
predictions = model.predict(spectrogram)

# Get the predicted class label
predicted_class = np.argmax(predictions, axis=-1)

# Print the predicted class label
print('Predicted class:', predicted_class)


import glob

# Search for the model file by name
model_files = glob.glob('**/music_updated.h5', recursive=True)

# Print the paths of all the matching files
for file in model_files:
    print("Model file found:", file)